<a href="https://colab.research.google.com/github/kellysolow/OFA-Chinese/blob/master/OFA_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content

!git clone https://github.com/yangjianxin1/OFA-Chinese.git

/content
Cloning into 'OFA-Chinese'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 162 (delta 50), reused 144 (delta 32), pack-reused 0
Receiving objects: 100% (162/162), 2.87 MiB | 35.87 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [ ]:
!pip install transformers==4.20.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 105.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ./OFA-Chinese

/content/OFA-Chinese


In [ ]:
!pip install loguru

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.4 MB/s eta 0:00:00


In [ ]:
#train_args/train_ofa.json -> model_name_or_path YeungNLP/ofa-cn-base-muge-v2
#-> train_caption_file   ../drive/MyDrive/NICE/train_caption.jsonl
#-> train_caption_file   ../drive/MyDrive/NICE/train_image.tsv
!CUDA_VISIBLE_DEVICES=0 python train.py --train_args_file train_args/train_ofa.json 

2023-04-03 08:18:59.547966: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 08:19:00.417059: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2023-04-03 08:19:12.634 | INFO     | __main__:main:51 - Total training params: 61.81M
2023-04-03 08:19:12.634 | INFO     | component.dataset:__init__:15 - loading data from:../drive/MyDrive/NICE/train_caption.jsonl and ../drive/MyDrive/NICE/trai

In [ ]:
#학습완료 후 저장위치 파악

In [ ]:
from component.ofa.modeling_ofa import OFAModelForCaption
from torchvision import transforms
from PIL import Image
from transformers import BertTokenizerFast
from tqdm import tqdm
import torch

model_name_or_path = './output/ofa/checkpoint-final'
csv_path = '../drive/MyDrive/NICE/'
val_set = pd.read_csv(f'{csv_path}/nice-val-5k.csv')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

image_file = './images/test/lipstick.jpg'


model = OFAModelForCaption.from_pretrained(model_name_or_path).to(device)
tokenizer = BertTokenizerFast.from_pretrained(model_name_or_path)


mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 256
patch_resize_transform = transforms.Compose([
        lambda image: image.convert("RGB"),
        transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

txt = " what does the image describe?"
inputs = tokenizer([txt], return_tensors="pt").input_ids.to(device)

resultDF = pd.DataFrame(columns = ['public_id', 'sentences'])


for pic_id in tqdm(val_set['public_id'], total = len(val_set)):
  img = Image.open(f'{csv_path}val/{pic_id}.jpg')
  patch_img = patch_resize_transform(img).unsqueeze(0).to(device)

  gen = model.generate(inputs, patch_images=patch_img, num_beams=5, no_repeat_ngram_size=3)
  
  gen_sentence = tokenizer.batch_decode(gen, skip_special_tokens=True)[0]
  
  resultDF = resultDF.append({'public_id':pic_id, 'sentences' :gen_sentence},ignore_index = True)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 50%|█████     | 2501/5000 [39:25<40:16,  1.03it/s]<ipython-input-25-25d5228a423e>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append({'public_id':pic_id, 'sentences' :gen_sentence},ignore_index = True)
 50%|█████     | 2502/5000 [39:26<40:03,  1.04it/s]<ipython-input-25-25d5228a423e>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append({'public_id':pic_id, 'sentences' :gen_sentence},ignore_index = True)
 50%|█████     | 2503/5000 [39:27<38:05,  1.09it/s]<ipython-input-25-25d5228a423e>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append({'public_id':pic_id, 'sentences' :gen_sentence},ignore_index = True)
 50%|█████

In [ ]:
resultDF.to_csv(f'{csv_path}/output/base_chinese.csv', encoding = 'utf-8', index = False)  

In [ ]:
resultDF

,public_id,sentences
0,216582038,view of a mountains
1,1590061574,a young woman looking at their
2,1868714447,a medium close up shot of a engineer standing ...
3,1813180760,low angle view of their
4,1578946151,low angle shot of a young man lying on a bench...
...,...,...
4995,1587836972,mother and sow while sitting in the foreground...
4996,217367066,view to their of theirfacturing on thesey of autu
4997,1571687297,view to their of their
4998,1297775183,a portrait shot of a businessman looking at th...


In [ ]:
!pip install pycocoevalcap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 17.4 MB/s eta 0:00:00


In [ ]:


from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.spice.spice import Spice


combined_df = resultDF.merge(val_set, on="public_id")

dict_gt = {}
dict_pred = {}
for idx, row in combined_df.iterrows():
  dict_gt[str(row["public_id"])] = [row["caption_gt"]]
  dict_pred[str(row["public_id"])] = [row["sentences"]]

score, _ = Bleu(4).compute_score(dict_gt, dict_pred)
idx = 1
for s in score:
  print(f'Bleu_{idx} : {s*100}')
  idx += 1

score, _ = Rouge().compute_score(dict_gt, dict_pred)
print(f'Rouge : {score*100}')

score, _ = Cider().compute_score(dict_gt, dict_pred)
print(f'Cider : {score*100}')

score, _ = Meteor().compute_score(dict_gt, dict_pred)
print(f'Meteor : {score*100}')

score, _ = Spice().compute_score(dict_gt, dict_pred)
print(f'Spice : {score*100}')

{'testlen': 39136, 'reflen': 56299, 'guess': [39136, 34136, 29184, 24280], 'correct': [9781, 2746, 866, 257]}
ratio: 0.6951455620881243
Bleu_1 : 16.119367929967414
Bleu_2 : 9.145101880397588
Bleu_3 : 5.429549357178526
Bleu_4 : 3.2331752491315653
Rouge : 17.19217532495273
Cider : 24.267090806889655
Meteor : 8.286507756110414
Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.
Spice : 11.60056411782628


In [ ]:
combined_df

,public_id,sentences,caption_gt,category
0,216582038,view of a mountains,View of snowy mountain range,outdoors
1,1590061574,a young woman looking at their,A young woman lying on a bed,misc
2,1868714447,a medium close up shot of a engineer standing ...,A horizontal view of a scientist in lab suit e...,office
3,1813180760,low angle view of their,A vertical shot of sunset on a beach,outdoors
4,1578946151,low angle shot of a young man lying on a bench...,A woman relaxing in a deck chair,outdoors
...,...,...,...,...
4995,1587836972,mother and sow while sitting in the foreground...,Father and young children cooking in kitchen,indoors
4996,217367066,view to their of theirfacturing on thesey of autu,Mountains and forest in Canada,outdoors
4997,1571687297,view to their of their,View to Ronne Bornholm Island Denmark,outdoors
4998,1297775183,a portrait shot of a businessman looking at th...,Young businessman in busy office looking at ca...,office
